In [1]:
import requests
import json
import time
import hashlib
import hmac
import base64

# Global variables

In [2]:
kchost = "https://api.kucoin.com"

In [3]:
with open("../../../keys/kc-key") as f:
    api_key, api_secret, api_passphrase = f.read().split()

# Util functions

In [4]:
def authenticate(endpoint, method, bodyStr=""):
    now = int(time.time() * 1000)
    str_to_sign = str(now) + method.upper() + endpoint + bodyStr
    signature = base64.b64encode(
        hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
    headers = {
        "KC-API-SIGN": signature,
        "KC-API-TIMESTAMP": str(now),
        "KC-API-KEY": api_key,
        "KC-API-PASSPHRASE": passphrase,
        "KC-API-KEY-VERSION": "2",
        "Content-Type": "application/json"
    }
    return headers

In [5]:
def paramURL(params):
    items = params.items()
    t = ["%s=%s" % (k, v) for k, v in items]
    return "&".join(t)
def getPrivate(endpoint, params):
    parmurl = paramURL(params)
    if len(parmurl) > 0:
        parmurl = "?" + parmurl
    endpoint += parmurl
    headers = authenticate(endpoint, "get")
    url = "%s%s" % (kchost, endpoint)
    return requests.get(url, headers=headers, timeout=5).json()

In [6]:
def postPrivate(endpoint, body):
    bodyStr = json.dumps(body)
    headers = authenticate(endpoint, "post", bodyStr)
    url = "%s%s" % (kchost, endpoint)
    return requests.post(url, headers=headers, data=bodyStr, timeout=5).json()

In [7]:
def delPrivate(endpoint, params):
    parmurl = paramURL(params)
    if len(parmurl) > 0:
        parmurl = "?" + parmurl
    endpoint += parmurl
    headers = authenticate(endpoint, "delete")
    url = "%s%s" % (kchost, endpoint)
    return requests.delete(url, headers=headers, timeout=5).json()

# Account info

In [11]:
endpoint = "/api/v1/accounts"
getPrivate(endpoint, {})

{'code': '200000',
 'data': [{'id': '620788520f6c7b000152b893',
   'currency': 'BNB',
   'type': 'trade',
   'balance': '0.2',
   'available': '0.2',
   'holds': '0'},
  {'id': '61ff9bac7d597e00017ab6ad',
   'currency': 'BNB',
   'type': 'main',
   'balance': '0',
   'available': '0',
   'holds': '0'}]}

In [8]:
json.dumps({"type": "main"})

'{"type": "main"}'

# Internal transfer

In [9]:
endpoint = "/api/v2/accounts/inner-transfer"
body = {
    "clientOid": str(int(time.time() * 1000)),
    "currency": "BNB",
    "from": "main",
    "to": "trade",
    "amount": 0.1
}

In [10]:
postPrivate(endpoint, body)

{'code': '200000', 'data': {'orderId': '6263cedd54d3dd0001dc2b3e'}}

# Sell order

In [27]:
endpoint = "/api/v1/orders"
body = {
    "clientOid": str(int(time.time() * 1000)),
    "side": "sell",
    "symbol": "BNB-USDT",
    "type": "limit",
    "price": 500,
    "size": 0.01
}

In [28]:
ret = postPrivate(endpoint, body)
orderid = ret['data']['orderId']

In [29]:
orderid

'6263d05ae6dd460001c28f69'

# Get order

In [20]:
endpoint = "/api/v1/orders/%s" % orderid
getPrivate(endpoint, {})

{'code': '200000',
 'data': {'id': '6263d000098e670001173967',
  'symbol': 'BNB-USDT',
  'opType': 'DEAL',
  'type': 'limit',
  'side': 'sell',
  'price': '404',
  'size': '0.02',
  'funds': '0',
  'dealFunds': '8.08944',
  'dealSize': '0.02',
  'fee': '0.00808944',
  'feeCurrency': 'USDT',
  'stp': '',
  'stop': '',
  'stopTriggered': False,
  'stopPrice': '0',
  'timeInForce': 'GTC',
  'postOnly': False,
  'hidden': False,
  'iceberg': False,
  'visibleSize': '0',
  'cancelAfter': 0,
  'channel': 'API',
  'clientOid': '1650708479064',
  'remark': None,
  'tags': None,
  'isActive': False,
  'cancelExist': False,
  'createdAt': 1650708480600,
  'tradeType': 'TRADE'}}

# Cancel order

In [16]:
endpoint = "/api/v1/orders/%s" % orderid
delPrivate(endpoint, {})

{'code': '200000', 'data': {'cancelledOrderIds': ['6263cf8aefa6c900012cc35f']}}

# Cancel all orders on a market

In [30]:
endpoint = "/api/v1/orders"
param = {
    "symbol": "BNB-USDT"
}
delPrivate(endpoint, param)

{'code': '200000',
 'data': {'cancelledOrderIds': ['6263d04d8bd161000133d0b8',
   '6263d05074b0a900012682a8',
   '6263d05ae6dd460001c28f69']}}